# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
from sqlalchemy import create_engine

import pandas as pd 
import numpy as np
import pickle
import nltk

nltk.download('punkt')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# load data from database
engine = create_engine('sqlite:///disasterdb.db')
df = pd.read_sql_table('disaster', engine)
X = df['message']
Y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']]

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline  = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
#target_names = Y.columns
def model_testing(y_test, y_pred):
    for i in range(36):
        print("classification report for " + y_test.columns[i]
              ,'\n', classification_report(y_test.values[:,i],y_pred[:,i])
              ,'\n accuracy:', accuracy_score(y_test.values[:, i], y_pred[:, i]))

        
model_testing(y_test, y_pred)

classification report for related 
              precision    recall  f1-score   support

          0       0.65      0.35      0.46      1556
          1       0.82      0.94      0.88      4964
          2       0.27      0.09      0.13        34

avg / total       0.78      0.80      0.77      6554
 
 accuracy: 0.795849862679
classification report for request 
              precision    recall  f1-score   support

          0       0.89      0.99      0.93      5462
          1       0.85      0.37      0.51      1092

avg / total       0.88      0.88      0.86      6554
 
 accuracy: 0.884040280745
classification report for offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554
 
 accuracy: 0.996185535551
classification report for aid_related 
              precision    recall  f1-score   support

          0       0.7

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=4)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total= 3.6min
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total= 3.6min
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total= 3.6min
[CV] clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1), total= 3.2min
[CV] clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 20.3min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 20.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
# predict on test data
y_pred = cv.predict(X_test)
model_testing(y_test, y_pred)
print("\nBest Parameters:", cv.best_params_)

classification report for related 
              precision    recall  f1-score   support

          0       0.62      0.32      0.42      1556
          1       0.81      0.94      0.87      4964
          2       0.17      0.03      0.05        34

avg / total       0.76      0.79      0.76      6554
 
 accuracy: 0.786695148001
classification report for request 
              precision    recall  f1-score   support

          0       0.89      0.98      0.94      5462
          1       0.84      0.39      0.54      1092

avg / total       0.88      0.89      0.87      6554
 
 accuracy: 0.88663411657
classification report for offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554
 
 accuracy: 0.996185535551
classification report for aid_related 
              precision    recall  f1-score   support

          0       0.72

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
svm_pipeline  = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state=0))))
    ])
svm_pipeline.fit(X_train, y_train)
y_pred = svm_pipeline.predict(X_test)
model_testing(y_test, y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


classification report for related 
              precision    recall  f1-score   support

          0       0.67      0.51      0.58      1556
          1       0.85      0.92      0.89      4964
          2       0.62      0.29      0.40        34

avg / total       0.81      0.82      0.81      6554
 
 accuracy: 0.81980469942
classification report for request 
              precision    recall  f1-score   support

          0       0.92      0.97      0.94      5462
          1       0.78      0.60      0.68      1092

avg / total       0.90      0.90      0.90      6554
 
 accuracy: 0.904943545926
classification report for offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554
 
 accuracy: 0.996185535551
classification report for aid_related 
              precision    recall  f1-score   support

          0       0.79

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [31]:
'''
with open('cv_model.pkl', 'wb') as file:
    pickle.dump(cv, file)
'''
with open('svm_pipeline_model.pkl', 'wb') as file:
    pickle.dump(svm_pipeline, file)   


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.